# This exercise will explore sparse representation from Section 3.6 to estimate a fluid flow field, following Callaham et al. [147]. Load the cylin- der flow data set. Coarsen each flow field by a factor of 20 in each direction using imresize, and build a library of these coarsened measurements (i.e., a matrix, where each column contains these downsampled measurements). Plot a movie of the flow field in these new coordinates. Now, pick a column of the full flow field matrix and add Gaussian random noise to this field. Downsam- ple the noisy field by a factor of 20 and use SRC to find the closest downsam- pled library element. Then use this column of the full flow field library as your reconstructed estimate.
# Try this approach with different levels of noise added to the original flow field. See how much noise is required before the method breaks. Try different ap- proaches to creating a low-dimensional representation of the image (i.e., in- stead of downsampling, you can measure the flow field in a small 5 × 10 patch and use this as the low-dimensional feature for SRC).

In [2]:
import numpy as np
import cv2
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
import scipy.io

# Load the cylinder flow data set (replace 'flow_data.npy' with your data file)
flow_data = scipy.io.loadmat('/Users/bhaskarspc/DATA_SVD_Assignment/FLUIDS/CYLINDER_ALL.mat')

# Coarsen the flow field by a factor of 20 in each direction using imresize
coarsened_flow = cv2.resize(flow_data, None, fx=1/20, fy=1/20, interpolation=cv2.INTER_AREA)

# Build a library of coarsened measurements
library = coarsened_flow.reshape(-1, coarsened_flow.shape[2]).T  # Each column contains a coarsened measurement




error: OpenCV(4.8.1) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from sklearn.linear_model import OrthogonalMatchingPursuit
import imageio

# Load the cylinder flow dataset (you need to replace this with your dataset loading code)
data = scipy.io.loadmat('/Users/bhaskarspc/DATA_SVD_Assignment/FLUIDS/CYLINDER_ALL.mat')

flow_field = data['UALL']

# Ensure the input array has at least one dimension
if flow_field.ndim == 0:
    flow_field = flow_field[None]

# Coarsen each flow field by a factor of 20 in each direction using imresize
coarsened_factor = 20
coarsened_flow = zoom(flow_field, (1 / coarsened_factor, 1 / coarsened_factor, 1), order=1)

# Build a library of these coarsened measurements
# Each column contains one coarsened flow field
library = coarsened_flow.reshape(-1, coarsened_flow.shape[-1]).T

# Plot a movie of the flow field in these new coordinates
def plot_flow_movie(flow_data, title="Flow Field Movie"):
    fig, ax = plt.subplots()
    ims = []
    for frame in range(flow_data.shape[-1]):
        im = ax.imshow(flow_data[:, :, frame], animated=True)
        ims.append([im])
    ani = imageio.mimwrite("flow_field_movie.gif", ims, duration=0.1)
    plt.show()

# plot_flow_movie(coarsened_flow, "Coarsened Flow Field Movie")

# Function to add Gaussian noise to a column of the full flow field matrix
def add_gaussian_noise(data, column_idx, std_dev):
    noisy_data = data.copy()
    noisy_data[:, column_idx] += np.random.normal(0, std_dev, data.shape[0])
    return noisy_data

# Downsample the noisy field by a factor of 20
def downsample_flow(noisy_data, factor):
    return noisy_data[::factor, ::factor]

# Use Sparse Representation (SRC) to find the closest downsampled library element
def sparse_representation_reconstruction(library, noisy_column):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=1)
    omp.fit(library, noisy_column)
    sparse_coefs = omp.coef_
    # Find the index of the best-matching library element
    best_match_index = np.argmax(sparse_coefs)
    return library[:, best_match_index]

# Perform SRC with different levels of noise
noise_levels = [0.01, 0.05, 0.1, 0.2]  # Adjust noise levels as needed

for noise_level in noise_levels:
    noisy_column = add_gaussian_noise(flow_field, selected_column_index, noise_level)
    downsampled_noisy_column = downsample_flow(noisy_column, coarsened_factor)
    reconstructed_column = sparse_representation_reconstruction(library, downsampled_noisy_column)
    
    # Visualize the results or compute error metrics as needed
    # You can calculate metrics like PSNR, SSIM, or other relevant measures to evaluate the reconstruction quality.


RuntimeError: sequence argument must have length equal to input rank

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Function to create a movie of the flow field
def create_flow_movie(flow_data, output_file, title="Flow Field Movie"):
    fig, ax = plt.subplots()
    ims = []
    for frame in range(flow_data.shape[-1]):
        im = ax.imshow(flow_data[:, :, frame], animated=True)
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, blit=True, interval=100)
    ani.save(output_file, writer="imagemagick", fps=10)

# Create a movie of the flow field in the new coordinates
create_flow_movie(coarsened_flow, "flow_field_movie.gif", "Coarsened Flow Field Movie")

# You can also display individual frames of the flow field
def display_frame(frame_data):
    plt.imshow(frame_data, cmap='viridis')
    plt.show()

# Display a specific frame (e.g., frame 0)
display_frame(coarsened_flow[:, :, 0])